In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import datetime
import random
import re
import pandas as pd
import numpy as np



def Get_detail_stage_ma(URL):
    html = urlopen(URL)
    bs_ = BeautifulSoup(html,'html.parser')
    description_entreprise = bs_.find('p',{'class':'text-justify objectifs-stagiaire'}).get_text()
    #print(description_entreprise)
    i = bs_.findAll('p')
    DESCRIPTION_ENTREPRISE=i[0].get_text()
    CONTEXTE_MISSION=i[1].get_text()
    PROFIL_RECHERCHE=i[2].get_text()
    return(DESCRIPTION_ENTREPRISE,CONTEXTE_MISSION,PROFIL_RECHERCHE)


def Get_stage_ma(PageNumber):
  
  html = urlopen('http://www.stage.ma/offres/{}'.format(PageNumber))
  bs = BeautifulSoup(html,'html.parser')
  data = {'Titre': [],'Company_name':[],'Stage_duree':[],'stage-depart':[],'stage-ville':[],'date-publication':[],'details':[]
          ,'DESCRIPTION-ENTREPRISE':[],'CONTEXTE-MISSION':[],'PROFIL-RECHERCHE':[]}
  Offres = bs.findAll('div',{'class':'offre-de-stage-block row'})
  for offre in Offres:
    Titre = offre.find('h5',{'class':'offre-stage-titre text-md-left text-center'}).get_text()
    data['Titre'].append(Titre)

    class_entreprise  = offre.findAll('a',{'class':'company-link'})
    for entreprise in class_entreprise :
      data['Company_name'].append(entreprise.get_text())

    stage_duree = offre.find('strong',{'class':'stage-duree'}).get_text()
    data['Stage_duree'].append(stage_duree)

    stage_depart = offre.find('span', {'class':'stage-depart'}).get_text()
    data['stage-depart'].append(stage_depart)

    stage_ville = offre.find('strong',{'class':'stage-ville'}).get_text()
    data['stage-ville'].append(stage_ville)

    date_publication = offre.find('span', {'class':'date-publication d-none d-md-block'}).get_text()
    data['date-publication'].append(date_publication)
    
    details_stage = offre.findAll('label',{'data-toggle':'tooltip'})
    des =''
    for detail in details_stage:
      des =des + detail.get_text() + '|'
    data['details'].append(des)
    
    link = offre.a['href']
    DESCRIPTION_ENTREPRISE,CONTEXTE_MISSION,PROFIL_RECHERCHE = Get_detail_stage_ma(str(link))
    data['DESCRIPTION-ENTREPRISE'].append(DESCRIPTION_ENTREPRISE)
    data['CONTEXTE-MISSION'].append(CONTEXTE_MISSION)
    data['PROFIL-RECHERCHE'].append(PROFIL_RECHERCHE)
    

   
  return pd.DataFrame(data)

pages = ['','10','20','30','40','50','60','70']
appended_data = []
for page in pages :
        
    data = Get_stage_ma(page)
    appended_data.append(data)
appended_data = pd.concat(appended_data)
 
#appended_data.head(20)



Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


In [4]:
appended_data.to_csv('StageMa.csv', index=False, encoding='utf-8')   

In [6]:
appended_data.to_excel('StageMa.xlsx', index=False, encoding='utf-8')